In [39]:
import pysam
import numpy as np

DIR = "/data/project/Meningioma/02.Align/hg38/Tumor/05.Final_bam/230405_Tumor.chr22.bam"

TUMOR_BAM_PYSAM = pysam.AlignmentFile( DIR , 'rb')
BLOOD_BAM_PYSAM = pysam.AlignmentFile( DIR.replace("Tumor", "Blood") , 'rb')

#1. TUMOR 의 read를 돌기
prev_pos = -1
cnt = 0
li_tumor, li_blood = [], []

for read in TUMOR_BAM_PYSAM.fetch():
    if (read.reference_name == "chr22") & ( read.pos > 29603000) & (read.pos < 29700000):
        if read.cigarstring != None:  # 왜 None이 있지?
            if ("S" in read.cigarstring)  |  ("H" in read.cigarstring):
                #print("Pos : {}\t{}\tRead name : {}\t".format(read.reference_name, read.pos, read.query_name))
                if read.pos != prev_pos:
                    cnt = 1
                    prev_pos = read.pos
                else:
                    cnt = cnt + 1

        
        if cnt  >= 2:
            li_tumor.append (prev_pos)
            cnt = 0

#2. BLOOD 의 read를 돌기
prev_pos = -1
cnt = 0

for read in BLOOD_BAM_PYSAM.fetch():
    if (read.reference_name == "chr22") & ( read.pos > 2960300) & (read.pos < 29700000):
        if read.cigarstring != None:  # 왜 None이 있지?
            if ("S" in read.cigarstring)  |  ("H" in read.cigarstring):
                #print("Pos : {}\t{}\tRead name : {}\t".format(read.reference_name, read.pos, read.query_name))
                if read.pos != prev_pos:
                    cnt = 1
                    prev_pos = read.pos
                else:
                    cnt = cnt + 1

        
        if cnt  >= 2:
            li_blood.append (prev_pos)
            cnt = 0


# Close the BAM file
TUMOR_BAM_PYSAM.close()
BLOOD_BAM_PYSAM.close()

In [93]:
import sys
import os
import pysam
import gzip
import numpy as np
import re


sample_name = [line.rstrip('\n') for line in open('/data/project/Meningioma/script/00.Targetted/sample_name.txt', 'r')]

start_end_list = []
f = open("/home/goldpm1/resources/NF2.exome.splicing.bed",'r')
for fline in f:
    ftab = fline.rstrip().split('\t')
    start = ftab[1]
    end = ftab[2]
    element = start + '_' + end
    start_end_list.append(element)
f.close()

reference = pysam.FastaFile('/home/goldpm1/reference/genome.fa')




for SAMPLE in sample_name:
    print (SAMPLE)

    bam_dir = '/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/05.Final_bam'
    
    samfile = pysam.AlignmentFile(bam_dir + "/" + SAMPLE + "_Dura.bam", "rb")
    dict = {}
    dict_mismatch = {}

    # INDEL
    for start_end in start_end_list:
        start_pos = int(start_end.rstrip().split('_')[0])
        end_pos = int(start_end.rstrip().split('_')[1])

        iter = samfile.fetch(contig = 'chr22', start = start_pos - 1, end = end_pos + 1)
        for i, read in enumerate(iter):
            if read.is_paired:
                read_info = str(read).rstrip().split('\t')
                        
                soft_al = read.query_alignment_start
                if read.cigarstring != None:  # 왜 None이 있지?
                    if ("D" in read.cigarstring)   |  ("I" in read.cigarstring) :
                        groups = re.findall(r"(\d+|\D+)", read.cigarstring)
                    else:
                        continue
                    
                    sum = 0 
                    if "D" in groups:
                        end = groups.index("D")
                    elif "I" in groups:
                        end = groups.index("I")

                    for i in range (0, end - 1 ):
                        if groups[i].isdigit():  # 숫자면 더해준다
                            sum = sum + int(groups[i])

                    if int (read.pos) + sum + 1  in dict.keys():
                        dict [ int (read.pos) + sum  + 1] += 1    # 있으면 그 key에 1씩 더해줌
                    else:
                        dict [ int (read.pos) + sum  + 1] = 1       # 없으면 1


    filtered_dict = {key: value for key, value in dict.items() if value  >= 2}
    print ( "INDEL >= 2 : ", filtered_dict  )

    
    # # 모든 read를 돌면서 multiallelic mismatch (variant)가 있는지 본다
    # for start_end in start_end_list:
    #     start_pos = int(start_end.rstrip().split('_')[0])
    #     end_pos = int(start_end.rstrip().split('_')[1])

    #     iter = samfile.fetch(contig = 'chr22', start = start_pos - 1, end = end_pos + 1)
    #     for i, read in enumerate(iter):  # 해당 splicing region의 read를 모두 소집
    #         reference_sequence = reference.fetch(read.reference_name, read.reference_start, read.reference_end)  # 그 read에 맞는 reference sequence

    #         i = 0
    #         for read_base, ref_base in zip(read.query_sequence, reference_sequence):
    #             if read_base != ref_base:
    #                 if read.reference_start + i in  dict_mismatch.keys():
    #                     dict_mismatch[read.reference_start + i].add ( read_base )  # set에 추가
    #                 else:
    #                     dict_mismatch[read.reference_start + i]  =  set(read_base)   # 1개짜리 set를 만들어서 넣어준다
    #             i = i + 1

    # if any(len(value) > 2 for value in dict_mismatch.values()) == True:
    #     dict_mismatch = {key:value for key, value in dict_mismatch.items() if len(value) >= 2 }
    



221021
INDEL >= 2 :  {29636686: 2, 29636922: 2, 29642294: 2, 29642318: 3, 29661208: 2, 29661237: 2, 29678244: 2}
221028
INDEL >= 2 :  {29639209: 2, 29658329: 2, 29665059: 2, 29668381: 3, 29668366: 2, 29683037: 6, 29683075: 6, 29683093: 6}
221202
INDEL >= 2 :  {29654693: 5, 29678332: 2}
230106
INDEL >= 2 :  {29603981: 4, 29639197: 3, 29639176: 2, 29661364: 2, 29682995: 3, 29683062: 3}
230127
INDEL >= 2 :  {29654671: 2, 29661417: 2, 29665067: 3, 29668289: 2, 29671853: 2, 29671960: 2}
230217
INDEL >= 2 :  {29639310: 2, 29642053: 2}
230303
INDEL >= 2 :  {29664999: 2, 29674746: 2, 29678154: 2, 29681461: 2, 29681596: 2}
230323
INDEL >= 2 :  {29661336: 75, 29668454: 4, 29668372: 3, 29681451: 2, 29682985: 3, 29683079: 6, 29683060: 6, 29683075: 3}


In [9]:
import pysam

# Open the BAM file
bam_file = pysam.AlignmentFile( '/data/project/Meningioma/00.Targetted/02.Align/hg38/Dura/05.Final_bam/221021_Dura.bam', 'r')

# Define the target position
chromosome = 'chr22'
position = 29639209

# Count the base distribution
base_counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}

for pileup_column in bam_file.pileup(chromosome, position - 1, position):
    if pileup_column.pos == position - 1:
        #print ( len( pileup_column.pileups )  )
        for pileup_read in pileup_column.pileups:
            if pileup_read.query_position == None:
                continue
            base = pileup_read.alignment.query_sequence[pileup_read.query_position]
            base_counts [base] += 1

print (base_counts)
bam_file.close()

{'A': 1144, 'C': 0, 'G': 0, 'T': 0}


### NF2 splicing region bed : /home/goldpm1/resources/NF2.exome.splicing.bed'

In [90]:
start_end_list = []
input_bed = open('/home/goldpm1/resources/NF2.exome.proteincoding.bed','r')
for fline in input_bed:
    ftab = fline.rstrip().split('\t')
    start = ftab[1]
    end = ftab[2]
    element_1 = str ( ( int(start) - 4) ) + '_' + str ( ( int(start) + 4) )
    element_2 = str ( ( int(end) - 4) ) + '_' + str ( ( int(end) + 4) )
    start_end_list.append(element_1)
    start_end_list.append(element_2)
input_bed.close()

start_end_list = sorted( list(set(start_end_list)))


output_file = open('/home/goldpm1/resources/NF2.exome.splicing.bed','w')
for line in start_end_list:
    print ("chr22\t{}\t{}\tsplicing_region".format (line.split("_")[0], line.split("_")[1]), file = output_file)
    #print ("chr22\t{}\t{}\tsplicing_region".format (line.split("_")[0], line.split("_")[1]))
output_file.close()



chr22	29603551	29603559	splicing_region
chr22	29603583	29603591	splicing_region
chr22	29603598	29603606	splicing_region
chr22	29603612	29603620	splicing_region
chr22	29603994	29604002	splicing_region
chr22	29604108	29604116	splicing_region
chr22	29628653	29628661	splicing_region
chr22	29628769	29628777	splicing_region
chr22	29636746	29636754	splicing_region
chr22	29636872	29636880	splicing_region
chr22	29639085	29639093	splicing_region
chr22	29639208	29639216	splicing_region
chr22	29642197	29642205	splicing_region
chr22	29642281	29642289	splicing_region
chr22	29654652	29654660	splicing_region
chr22	29654721	29654729	splicing_region
chr22	29654970	29654978	splicing_region
chr22	29655076	29655084	splicing_region
chr22	29658184	29658192	splicing_region
chr22	29658260	29658268	splicing_region
chr22	29661200	29661208	splicing_region
chr22	29661335	29661343	splicing_region
chr22	29664985	29664993	splicing_region
chr22	29665060	29665068	splicing_region
chr22	29668328	29668336	splicing_region
